In [6]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

### Importamos los dataset

In [2]:
train=pd.read_csv("./Input/clean_train.csv")
pred=pd.read_csv("./Input/clean_pred.csv")

In [3]:
train.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,0.50,1,0,4,62.3,55.0,5.11,5.07,3.17,1845
1,1,1.54,3,5,3,63.6,60.0,7.30,7.33,4.65,10164
2,2,1.32,1,6,6,61.7,60.0,6.95,7.01,4.31,5513
3,3,1.20,1,5,5,62.1,55.0,6.83,6.79,4.23,5174
4,4,1.73,0,5,5,61.2,60.0,7.67,7.65,4.69,10957


In [4]:
pred.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.45,0,1,5,62.8,58.0,4.88,4.84,3.05
1,1,1.23,1,4,5,61.0,56.0,6.96,6.92,4.23
2,2,0.33,1,5,0,61.8,55.0,4.46,4.47,2.76
3,3,0.51,0,0,4,58.0,60.0,5.29,5.26,3.06
4,4,0.40,0,1,4,62.2,59.0,4.71,4.74,2.94


### Eliminamos la columna 'price'

In [8]:
train_clean = train[['carat','cut','color','clarity','depth','table', 'x', 'y' , 'z']]

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test  = train_test_split(train_clean ,train.price, test_size=0.2)

train_split=[X_train, X_test, y_train, y_test ]

In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(32364, 9)
(32364,)
(8091, 9)
(8091,)


### Importamos todos los metodos posibles para realizar predicciones

In [13]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error

### Creamos un diccionario con los metodos que queremos testear

In [14]:
models = {
    "forest" : RandomForestRegressor(),
    #"tree" : DecisionTreeRegressor(),
    #"neighbors_reg": KNeighborsRegressor(),
    #"gradient": GradientBoostingRegressor()
    }

###### Esto tarda muchisimo y satura mi PC, por lo que para el ejemplo solo uso dos metodos

In [15]:
for name, model in models.items():
    print(f"Training {name}")
    model.fit(X_train, y_train)
print("Finish")

Training forest
Training tree
Finish


###### Como vemos en este caso el metodo con más adecuado es RandomForestRegressor()

In [16]:
from sklearn.metrics import mean_squared_error
for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"-------{name}-------")
    print ("RMSE", round(np.sqrt(mean_squared_error(y_test,y_pred)),3))

-------forest-------
RMSE 540.365
-------tree-------
RMSE 751.255


In [21]:
import numpy as np
from sklearn.model_selection import cross_val_score

In [22]:
for name, model in models.items():
    scores = cross_val_score(model,train_clean ,train.price, cv=10)
    print(f"{name} accuracy: {np.mean(scores)}")

forest accuracy0.9813076134179681
tree accuracy0.9639390717248901


#### Creamos un diccionario con diferentes posibles valores para configurar RandomForestRegressor()

In [17]:
parameters = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200]}

#### Creamos la variable con esta función

In [18]:
rfr = RandomForestRegressor()

In [19]:
rfr.fit(X_train,y_train)

RandomForestRegressor()

#### La libreria "GridSearchCV" realiza un ajuste de hiperparámetros para determinar los valores óptimos para un modelo en concreto

In [20]:
from sklearn.model_selection import GridSearchCV

#### El parametro "n_jobs" es el número de trabajos a ejecutar en paralelo. Para ejecutarlos todos hay que escribir "n_jobs=-1".
#### Yo lo hice en su momento pero tardó varias horas en completar, por lo que no lo volveré a ejecutar.

In [ ]:
grid = GridSearchCV(rfr, params, n_jobs=-1, verbose=1)
grid.fit(X_train,y_train)

In [ ]:
print(grid.best_params_)
# Output: {'max_depth': 10, 'max_features': 50, 'min_samples_leaf': 20, 'min_samples_split': 15, 'n_estimators': 50}